In [ ]:
######################################################################

# I acknowledge the use of AI tools in debugging and writing the code

######################################################################

In [3]:
import pandas as pd
file_path = "/Users/marcosmaldacena/Downloads/Thesis Data/merged_earnings_call_transcripts.csv"
transcripts = pd.read_csv(file_path)

In [6]:
print(transcripts[transcripts['ticker'] == 'DELL'])

      ticker  year  quarter                 date  \
0       DELL  2015        1  2015-04-22 15:09:05   
1       DELL  2015        2  2015-07-22 15:42:09   
2       DELL  2015        3  2015-10-21 13:20:18   
3       DELL  2015        4  2016-01-27 15:19:05   
4       DELL  2016        1  2016-04-20 15:53:15   
3691    DELL  2014        1  2014-04-23 14:21:06   
3692    DELL  2014        2  2014-07-23 15:32:03   
3693    DELL  2014        3  2014-10-22 15:06:03   
3694    DELL  2014        4  2015-01-30 16:24:09   
5521    DELL  2017        3  2016-12-08 11:31:07   
5522    DELL  2017        4  2017-03-30 15:29:04   
5523    DELL  2018        1  2017-06-08 14:52:06   
5524    DELL  2018        2  2017-09-07 12:23:06   
5525    DELL  2018        3  2017-12-07 13:07:05   
5526    DELL  2018        4  2018-03-08 14:58:03   
5527    DELL  2019        1  2018-06-04 15:16:06   
5528    DELL  2019        2  2018-09-06 14:45:06   
5529    DELL  2019        3  2018-11-29 23:47:04   
5530    DELL

In [2]:
!pip install openai neo4j


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.1 kB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 16.8 MB/s eta 0:00:00


In [6]:
from neo4j import GraphDatabase
import openai

# Neo4j Connection Setup
class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def create_relationships(self, data):
        with self.driver.session() as session:
            for row in data:
                session.write_transaction(self._create_entity_relationship, row)
    
    @staticmethod
    def _create_entity_relationship(tx, row):
        query = """
        MERGE (e:Entity {name: $entity})
        MERGE (t:Target {name: $target})
        MERGE (e)-[:RELATIONSHIP {type: $relationship}]->(t)
        """
        tx.run(query, entity=row["entity"], relationship=row["relationship"], target=row["target"])


# GPT-4o API Wrapper
def generate_gpt_output(prompt, api_key):
    openai.api_key = api_key
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Replace with "gpt4o-mini" if needed
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    # Extract relationships from the GPT response
    # Assuming GPT output is structured like: "Entity | Relationship | Target"
    output = response.choices[0].message.content
    relationships = []
    for line in output.strip().split("\n"):
        try:
            entity, relationship, target = map(str.strip, line.split("|"))
            relationships.append({"entity": entity, "relationship": relationship, "target": target})
        except ValueError:
            print(f"Skipping malformed line: {line}")
    return relationships


def main():
    # Neo4j connection details
    uri = "bolt://localhost:7687"  # Update with your Neo4j instance URI
    user = "neo4j"                # Default username
    password = "Macocho18!"       # Replace with your Neo4j password

    # Initialize Neo4j handler
    neo4j_handler = Neo4jHandler(uri, user, password)

    try:
        # Your GPT-4 output
        entity = "Dell Technologies"
        relationship = "reports"
        target = "Q3 $22.3 billion"
        
        # Wrap the single dictionary in a list
        gpt_output = [{"entity": entity, "relationship": relationship, "target": target}]
        
        # Insert data into Neo4j
        neo4j_handler.create_relationships(gpt_output)
        print("Data successfully populated into Neo4j!")
    
    finally:
        # Close Neo4j connection
        neo4j_handler.close()


if __name__ == "__main__":
    main()

/var/folders/m2/ny_3r2zn14z8qxzlqbt2q1dh0000gn/T/ipykernel_13206/3510127673.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_entity_relationship, row)


Data successfully populated into Neo4j!
